# LDAの動作確認 part 2

LDA_in_scikit_learn_00 での動作確認結果が、いまひとつ理解できなかったため、公式ドキュメントを確認しコメントを入れながら、再度調査してみました。

公式ドキュメント：

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

結論から申し上げると、

(1) 使用した以下のサンプル（公式ドキュメントのサンプル）は、辞書なし学習の例でした。

(2) LatentDirichletAllocation というモジュールの仕様上、学習実行--->即結果が出力される・・・といった使い勝手のようです。

このレポートでは、公式ドキュメントから読み取れる仕様にもとづき、正しいと思われる使い方で再試行してみました。

## テスト用データ（2,000行）のロード

In [226]:
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, 
                             random_state=1,
                             remove=('headers', 'footers', 'quotes'))
test_data = dataset.data[:2000]

print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 1.674s.


In [227]:
len(test_data)

2000

## コーパスの作成

In [228]:
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95,
                                min_df=3,
                                max_features=1000, # 特徴語の上限
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(test_data)
print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 0.319s.


2,000 件のサンプルテキストから、51,754 件の単語が抽出され、戻り値の tf に単語の出現回数がセットされています。

In [229]:
len(tf.data)

51754

In [230]:
tf.data[0:99]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 2, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [231]:
tf_feature_names = tf_vectorizer.get_feature_names()
len(tf_feature_names)

1000

## 学習実行

In [232]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=10, # １０のトピックに分類
                                max_iter=20, # ２０回繰り返し実行
                                learning_method='online', # 学習実行-->即結果を出力
                                learning_offset=50.,
                                random_state=1)
t0 = time()
answers = lda.fit_transform(tf)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=1000 and n_features=500...
done in 8.014s.


## 学習結果

１０のトピックに含まれる上位１０件の特徴語をリストします。

In [233]:
n_top_words = 10
top_words = []
for topic_idx, topic in enumerate(lda.components_):
    print("Topic #%d:" % topic_idx)
    top_words_idx = topic.argsort()[:(-n_top_words - 1):-1]
    top_words_list = [tf_feature_names[i] for i in top_words_idx]
    top_words.append(top_words_list)
    print(",".join(top_words_list))

Topic #0:
edu,com,mail,send,ftp,graphics,pub,message,contact,faq
Topic #1:
10,55,11,space,team,12,15,18,game,20
Topic #2:
god,jesus,bible,church,people,christian,does,hiv,faith,christ
Topic #3:
law,section,weapons,gun,military,weapon,shall,firearm,person,dangerous
Topic #4:
drive,disk,hard,drives,scsi,controller,card,rom,bios,16
Topic #5:
car,key,government,000,chip,people,encryption,year,new,clipper
Topic #6:
bike,cubs,win,dog,runs,games,game,ground,division,head
Topic #7:
israel,jews,jewish,women,men,attacks,world,war,israeli,price
Topic #8:
use,windows,thanks,program,using,software,new,version,pc,computer
Topic #9:
just,don,people,like,think,know,good,time,way,say


トピックの分類結果を表示します。

試しに、テストデータ #557 で見てみます。

In [234]:
answers[557]

array([ 0.01000431,  0.01000377,  0.01000074,  0.01000024,  0.25526706,
        0.01000225,  0.01000087,  0.01000154,  0.66471858,  0.01000063])

９番目のトピック #8 との回答です。テストデータの内容と突合すると、当たっているようです。

In [235]:
print("==========")
print("Topic #8:")
print(",".join(top_words[8]))
print("==========")
print("Sample Text:")
print(data_samples[557])

Topic #8:
use,windows,thanks,program,using,software,new,version,pc,computer
Sample Text:
Is there a QIC-80 format tape drive that comes
with an EISA controller ?
Colorado's 250 only has ISA and MCA controllers.

Thanks. e-mail please.

-- 
